In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [26]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [7]:
base  = '/content/drive/My Drive/Airwritten Dataset'
images = []
labels = []

for label in os.listdir(base):
  ok = os.path.join(base, label)
  for f in os.listdir(ok):
    print(os.path.join(ok, f))
    img = cv2.imread(os.path.join(ok, f))
    resized = cv2.resize(img, (96, 96))
    resized = resized / 255.0
    images.append(resized)
    labels.append(label)

/content/drive/My Drive/Airwritten Dataset/ka/0eb01664-c895-4421-bd3a-b7eaecbdd06a.jpg
/content/drive/My Drive/Airwritten Dataset/ka/1fc15ca0-cffa-48c3-8dc3-544bf6bf757f.jpg
/content/drive/My Drive/Airwritten Dataset/ka/0e0ef2dd-3aa6-4209-8348-dec53178769b.jpg
/content/drive/My Drive/Airwritten Dataset/ka/eaa0b08f-2950-4719-a1ae-4ab472b4434a.jpg
/content/drive/My Drive/Airwritten Dataset/ka/d2563527-e6e3-4f02-98e0-6423e77e6774.jpg
/content/drive/My Drive/Airwritten Dataset/ka/d18daef6-cf3f-432d-bb59-678b2d88f363.jpg
/content/drive/My Drive/Airwritten Dataset/ka/daf9008f-d9f9-4bb4-9ce6-fc143c775894.jpg
/content/drive/My Drive/Airwritten Dataset/ka/d65cb160-8482-4fb1-8bc3-38fb9e5b6860.jpg
/content/drive/My Drive/Airwritten Dataset/ka/d13827b7-9059-48d1-b125-4655a4c827fa.jpg
/content/drive/My Drive/Airwritten Dataset/ka/dbe33992-3223-42d1-a769-a36ec9ad4cff.jpg
/content/drive/My Drive/Airwritten Dataset/ka/ee65e3a1-189c-449a-a11f-17379642755e.jpg
/content/drive/My Drive/Airwritten Dataset/

In [ ]:
# class_names = sorted([label for label in os.listdir(base)])

In [27]:
npa_images = np.array(images)
npa_labels = np.array(labels)
X = npa_images
y = npa_labels

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# convert to one-hot for neural nets
y_categorical = to_categorical(y_encoded)

In [47]:
# y_categorical[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [32]:
class_names = label_encoder.classes_

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, random_state=11, test_size=0.2)

In [29]:
base_model = MobileNetV2(input_shape=(96, 96, 3),
                         include_top=False,
                         weights='imagenet')
base_model.trainable = False

In [30]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
output = Dense(len(class_names), activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=output)

In [31]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [35]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=32)

Epoch 1/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 13s 331ms/step - accuracy: 0.1011 - loss: 3.9786 - val_accuracy: 0.6186 - val_loss: 1.7405
Epoch 2/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 9s 345ms/step - accuracy: 0.5764 - loss: 1.6292 - val_accuracy: 0.8454 - val_loss: 0.8198
Epoch 3/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 238ms/step - accuracy: 0.8159 - loss: 0.8204 - val_accuracy: 0.9536 - val_loss: 0.4590
Epoch 4/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 11s 261ms/step - accuracy: 0.8987 - loss: 0.4911 - val_accuracy: 0.9639 - val_loss: 0.3391
Epoch 5/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 11s 273ms/step - accuracy: 0.9498 - loss: 0.3340 - val_accuracy: 0.9794 - val_loss: 0.2480
Epoch 6/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 11s 309ms/step - accuracy: 0.9438 - loss: 0.2808 - val_accuracy: 0.9794 - val_loss: 0.2029
Epoch 7/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 230ms/step - accuracy: 0.9711 - loss: 0.1961 - val_accuracy: 0.9897 - val_loss: 0.1693
Epoch 8/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 10s 230ms/step - accuracy: 0.9781 - loss: 0.1721 - val

In [36]:
model.save('/content/drive/My Drive/mobilenetv2_burmese.h5')

In [37]:
model.save('/content/drive/My Drive/mobilenetv2_burmese.keras')

In [48]:
# label_encoder.classes_

array(['a', 'ba', 'bha', 'da1', 'da2', 'dha1', 'dha2', 'ga', 'gha', 'ha',
       'hpa', 'hsa', 'hta1', 'hta2', 'ka', 'kha', 'la', 'la2', 'ma',
       'na1', 'na2', 'nga', 'nya', 'pa', 'ra', 'sa1', 'ta1', 'ta2', 'tha',
       'wa', 'ya', 'za', 'zha'], dtype='<U4')

In [51]:
os.makedirs('/content/drive/My Drive/MobileNetV2 Models/labels.txt', exist_ok=True)

FileExistsError: [Errno 17] File exists: '/content/drive/My Drive/labels.txt'

In [50]:
with open("/content/drive/My Drive/MobileNetV2 Models/labels.txt", "w", encoding='utf-8') as f:
    for index, name in enumerate(label_encoder.classes_):
        f.write(f'{index} {name}' + "\n")